In [36]:
__author__ = "Chuhang Liu"

import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
import scipy.signal
import scipy
from matplotlib import figure
import h5py
import hdf5plugin
import cv2


In [31]:
# Define source dict
dirstr = 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\'
# Define working dict
dest_dir = 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\im\\'

In [32]:
import glob

folder_path = dirstr
file_extension = '*.h5'

file_list = glob.glob(folder_path + file_extension)

# Find the file with keyword
keyword = 'master'
# Remove the file extension
file_list = [file.rstrip('.h5') for file in file_list if keyword in file]
print(file_list)




['D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_1mm_x4k_2s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_200um_x4k_2s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_500um_lx6000_1s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_500um_x4k_2s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_n1mm_x4k_2s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_n200um_x4k_2s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_n500nm_lx6000_1s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\defocus_n500um_x4k_2s_000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\def_1mm_x3000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\def_n1mm_x3000_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\focus_lx6000_1s_000h5_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\focus_lx6000_1s_002_master', 'D:\\2100f_data_DECTRIS\\ASI_S1_10um_20240305\\foc

In [38]:
for file in file_list:
    with h5py.File(file + '.h5', 'r') as f:
        #print(f.keys())
        
        # Load data
        dataset = f['entry/data/data_000001']
        # Read the dataset into a NumPy array
        data = np.array(dataset)
        # For images with dead pixels
        #data[data == 4294967295] = 0
        data = data.reshape(512, 512)
        # generate tiffs
        name = file.split('\\')[-1]
        try:
            plt.imsave(dest_dir + file + '.tif', data, format='tiff', cmap=plt.cm.gray)
        except:
            os.makedirs(dest_dir, exist_ok=True)
            plt.imsave(dest_dir + name + '.tif', data, format='tiff', cmap=plt.cm.gray)